# Prequesties

## Load data into dataframe 

In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
    header=True,
    inferSchema=True
)
df.display()

## Transaction 01 - Write data as Delta format

In [0]:
df_delta = df.write.save(
    format="DELTA",
    mode="overwrite",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta",
)
display(df_delta)

## Transaction 02

In [0]:
from pyspark.sql.functions import col
df.filter(col("country") == "India").write.save(
    format="DELTA",
    mode="overwrite",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta",
)

## Read the latest version by Default

In [0]:
spark.read.format("delta").load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()

## Desplay based on Version 

In [0]:
spark.read.option("versionAsOf",0).load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()

# Read Transaction Log

## Approach 1

In [0]:
spark.read.load(
    format="text",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/_delta_log/00000000000000000000.json",
).display()

In [0]:
spark.read.load(
    format="text",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/_delta_log/00000000000000000001.json",
).display()

## Approach 2

In [0]:
from delta.tables import DeltaTable
DeltaTable.forPath(spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").history().display()

# Transaction 03 - Updates are allowed
ADVANTAGE
1. Read Delta
2. Create View 
3. Update a View

In [0]:
df_delts_sec = spark.read.format("delta").load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")
df_delts_sec.display()

In [0]:
df_delts_sec.createOrReplaceTempView("user_sec_vw")

In [0]:
%sql
UPDATE
  user_sec_vw
SET
  city = 'Bengaluru'
WHERE
  city = "Bangalore"

In [0]:
%sql

SELECT * from user_sec_vw